In [1]:
!git clone https://github.com/fastai/course-v3/

Cloning into 'course-v3'...
remote: Enumerating objects: 5893, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 5893 (delta 0), reused 2 (delta 0), pack-reused 5890
Receiving objects: 100% (5893/5893), 263.03 MiB | 21.55 MiB/s, done.
Resolving deltas: 100% (3249/3249), done.


In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd /content/course-v3/nbs/dl2/
from exp.nb_06 import *

/content/course-v3/nbs/dl2


In [2]:
def get_data():
    # path = datasets.download_data(MNIST_URL, ext='.gz')
    path = '/content/mnist.pkl.gz'
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

In [3]:
x_train, y_train, x_valid, y_valid = get_data()
x_train, x_valid = normalize_to(x_train, x_valid)

train_ds, valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid)
nh, bs = 50, 512
c = y_train.max().item()+1
loss_func = F.cross_entropy

data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)
mnist_view = view_tfm(1, 28, 28)
cbfs = [Recorder,
        partial(AvgStatsCallback, accuracy),
        partial(BatchTransformXCallback, mnist_view)]

nfs = [8, 16, 32, 64, 64]
learn, run = get_learn_run(nfs, data, 0.4, conv_layer, cbs=cbfs)

# %time run.fit(4, learn)

In [12]:
??get_cnn_model
# ??get_learn_run

In [ ]:
??Lambda

In [7]:
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func
    def forward(self, x): return self.func(x)

In [5]:
def mnist_view(x): return x.view(-1, 1, 28, 28)

In [6]:
def flatten(x): return x.view(x.shape[0], -1)

In [7]:
def get_cnn_model(data):
    return nn.Sequential(
        Lambda(mnist_view),
        nn.Conv2d(in_channels=1, out_channels=8, kernel_size=5, stride=2, padding=2), nn.ReLU(),
        nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.AdaptiveAvgPool2d(1),
        Lambda(flatten),
        nn.Linear(32, data.c)
    )

In [14]:
xb, yb = next(iter(data.train_dl))
get_cnn_model(data)(xb).shape

torch.Size([512, 10])

---

In [11]:
class CudaCallback(Callback):
    def begin_fit(self):
        self.model = self.model.cuda()
    def begin_batch(self):
        self.run.xb, self.run.vb = self.xb.cuda(), self.yb.cuda()
        

In [8]:
loss_func = F.cross_entropy
data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)
model = get_cnn_model(data)
opt = optim.SGD(model.parameters(), lr=0.4)
learn = Learner(model, opt, loss_func, data)

cbfs = [Recorder,
        partial(AvgStatsCallback, accuracy),
        CudaCallback,
        partial(BatchTransformXCallback, mnist_view)]


run = Runner(cb_funcs=cbfs)
%time run.fit(3, learn)

train: [2.31050140625, tensor(0.1333, device='cuda:0')]
valid: [2.2949103515625, tensor(0.1064, device='cuda:0')]
train: [1.46553796875, tensor(0.5190, device='cuda:0')]
valid: [0.34649287109375, tensor(0.9001, device='cuda:0')]
train: [0.27712203125, tensor(0.9160, device='cuda:0')]
valid: [0.236508251953125, tensor(0.9326, device='cuda:0')]
CPU times: user 4.45 s, sys: 888 ms, total: 5.34 s
Wall time: 5.26 s
